In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, auc, roc_curve
from sklearn.preprocessing import RobustScaler
import time
import pickle
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
#reading csv file for the filtered data
columns = ["AccelX(g)_leg","AccelY(g)_leg","AccelZ(g)_leg","GyroX(¬∞/s)_leg","GyroY(¬∞/s)_leg","GyroZ(¬∞/s)_leg","AccelX(g)_hand","AccelY(g)_hand","AccelZ(g)_hand","GyroX(¬∞/s)_hand","GyroY(¬∞/s)_hand","GyroZ(¬∞/s)_hand","AccelX(g)_chest","AccelY(g)_chest","AccelZ(g)_chest","Sample(V)","Activity_Label"]
df = pd.read_csv("resampled_data.csv", header=0, names = columns)

In [3]:
PredictorCol=['AccelX(g)_leg', 'AccelY(g)_leg', 'AccelZ(g)_leg', 'GyroX(¬∞/s)_leg',
       'GyroY(¬∞/s)_leg', 'GyroZ(¬∞/s)_leg', 'AccelX(g)_hand',
       'AccelY(g)_hand', 'AccelZ(g)_hand', 'GyroX(¬∞/s)_hand',
       'GyroY(¬∞/s)_hand', 'GyroZ(¬∞/s)_hand', 'AccelX(g)_chest',
       'AccelY(g)_chest', 'AccelZ(g)_chest', 'Sample(V)']
TargetCol= 'Activity_Label'

X = df[PredictorCol].values
y = df[TargetCol].values

In [4]:
X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X, y, test_size=0.30, 
                                                    random_state=15, 
                                                    stratify = None)

In [6]:
from keras.layers import LSTM, Dense
from keras.models import Sequential

# Reshape the data for LSTM
X_train_sm = np.reshape(X_train_sm, (X_train_sm.shape[0], 1, X_train_sm.shape[1]))
X_test_sm = np.reshape(X_test_sm, (X_test_sm.shape[0], 1, X_test_sm.shape[1]))

# Initialize the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(1, X_train_sm.shape[2]), return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(16, activation='relu'))
model.add(Dense(len(np.unique(y)), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_sm, y_train_sm, epochs=10, batch_size=32)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test_sm, y_test_sm)

print("Test Accuracy:", test_accuracy)


Epoch 1/10
92224/92224 [==============================] - 263s 3ms/step - loss: 0.8962 - accuracy: 0.6978
Epoch 2/10
92224/92224 [==============================] - 265s 3ms/step - loss: 0.6694 - accuracy: 0.7714
Epoch 3/10
92224/92224 [==============================] - 267s 3ms/step - loss: 0.6189 - accuracy: 0.7890
Epoch 4/10
92224/92224 [==============================] - 267s 3ms/step - loss: 0.5929 - accuracy: 0.7978
Epoch 5/10
92224/92224 [==============================] - 268s 3ms/step - loss: 0.5759 - accuracy: 0.8033
Epoch 6/10
92224/92224 [==============================] - 268s 3ms/step - loss: 0.5647 - accuracy: 0.8072
Epoch 7/10
92224/92224 [==============================] - 269s 3ms/step - loss: 0.5562 - accuracy: 0.8097
Epoch 8/10
92224/92224 [==============================] - 270s 3ms/step - loss: 0.5496 - accuracy: 0.8122
Epoch 9/10
92224/92224 [==============================] - 271s 3ms/step - loss: 0.5438 - accuracy: 0.8140
Epoch 10/10
39525/39525 [=====================

In [7]:
model.save("model.h5")

In [8]:
import pickle

# Save model architecture to a file
with open('LSTMmodel_architecture.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save model weights to a file
with open('LSTMmodel_weights.pkl', 'wb') as f:
    pickle.dump(model.get_weights(), f)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......lstm
.........cell
............vars
...............0
...............1
...............2
.........vars
......lstm_1
.........cell
............vars
...............0
...............1
...............2
.........vars
......lstm_2
.........cell
............vars
...............0
...............1
...............2
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
.........21
.........22
.........23
.........24
.........25
.........26
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model

#To load the pickled model, you can use the following code:

import pickle

# Load model architecture
with open('model_architecture.pkl', 'rb') as f:
    model = pickle.load(f)

# Load model weights
with open('model_weights.pkl', 'rb') as f:
    model.set_weights(pickle.load(f))


